In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import trainer
from libs import freeze

W0110 18:31:30.252406 20252 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0110 18:31:30.252406 20252 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.



In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','bed,bird,cat,dog,down,eight,five,four,go,happy,house,left,marvin,nine,no,off,on,one,right,seven,sheila,six,stop,three,tree,two,up,wow,yes,zero','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',30,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',10.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [4]:
model_architecture='conv'
start_checkpoint=None
logging_interval=1
eval_step_interval=2000
save_step_interval=1
silence_percentage=10.0
unknown_percentage=10.0
background_frequency=0.8
background_volume=0.1
learning_rate='0.001,0.0001' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='15000,300' #Declare  the training steps for which the learning rates will be used
batch_size=100

In [5]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_data, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings, background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_data, train_ground_truth

def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_data, validation_ground_truth = (
        audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_data,validation_ground_truth

In [6]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [7]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [8]:
# WORD_LIST = [x[0].rsplit('\\', 2)[-1] for x in os.walk(train_dir)]
# ','.join(WORD_LIST)

In [9]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings)

..\..\_inputs\raw\train\audio\*\*.wav


W0110 18:32:05.313956 20252 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0110 18:32:05.327412 20252 deprecation_wrapper.py:119] From ..\libs\input_data.py:306: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 98, 257), dtype=float32)


In [10]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    data_size = model_settings['data_size']
    label_count = model_settings['label_count']
    data_input = tf.placeholder(
      tf.float32, [None, data_size], name='data_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    # Create Model
    
    logits, dropout_prob = models.create_model(
      data_input,
      model_settings,
      model_architecture,
      is_training=True)
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,data_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [11]:
tf.app.run(main=main)

W0110 18:32:08.146745 20252 deprecation_wrapper.py:119] From ..\libs\models.py:327: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0110 18:32:08.159743 20252 deprecation.py:506] From ..\libs\models.py:335: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0110 18:32:08.172709 20252 deprecation_wrapper.py:119] From ..\libs\models.py:338: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0110 18:32:08.204852 20252 deprecation.py:323] From ..\libs\trainer.py:57: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.n

I0110 18:33:35.501662 20252 trainer.py:126] Step #26: rate 0.001000, accuracy 10.0%, cross entropy 3.383437
I0110 18:33:35.501662 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-26"
I0110 18:33:38.563716 20252 trainer.py:126] Step #27: rate 0.001000, accuracy 13.0%, cross entropy 3.248640
I0110 18:33:38.563716 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-27"
I0110 18:33:41.739158 20252 trainer.py:126] Step #28: rate 0.001000, accuracy 22.0%, cross entropy 2.997142
I0110 18:33:41.740155 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-28"
I0110 18:33:45.414526 20252 trainer.py:126] Step #29: rate 0.001000, accuracy 10.0%, cross entropy 3.275218
I0110 18:33:45.415524 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-29"
I0110 18:33:48.484087 20252 trainer.py:126] Step #30: rate 0.001000, accuracy 20.0%, cross entropy 3.104530
I0110 18:33:48.484087 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-30"
I0110

I0110 18:35:42.783963 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-67"
I0110 18:35:45.793504 20252 trainer.py:126] Step #68: rate 0.001000, accuracy 20.0%, cross entropy 2.795609
I0110 18:35:45.793504 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-68"
I0110 18:35:48.796115 20252 trainer.py:126] Step #69: rate 0.001000, accuracy 22.0%, cross entropy 2.933894
I0110 18:35:48.796115 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-69"
I0110 18:35:52.226528 20252 trainer.py:126] Step #70: rate 0.001000, accuracy 22.0%, cross entropy 2.866233
I0110 18:35:52.226528 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-70"
I0110 18:35:55.192498 20252 trainer.py:126] Step #71: rate 0.001000, accuracy 18.0%, cross entropy 2.624327
I0110 18:35:55.192498 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-71"
I0110 18:35:58.161331 20252 trainer.py:126] Step #72: rate 0.001000, accuracy 20.0%, cross entropy 2.794408
I0110

I0110 18:37:54.837335 20252 trainer.py:126] Step #109: rate 0.001000, accuracy 28.0%, cross entropy 2.680964
I0110 18:37:54.839332 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-109"
I0110 18:37:58.144311 20252 trainer.py:126] Step #110: rate 0.001000, accuracy 27.0%, cross entropy 2.639969
I0110 18:37:58.144311 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-110"
I0110 18:38:01.825906 20252 trainer.py:126] Step #111: rate 0.001000, accuracy 25.0%, cross entropy 2.417735
I0110 18:38:01.827895 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-111"
I0110 18:38:05.529174 20252 trainer.py:126] Step #112: rate 0.001000, accuracy 39.0%, cross entropy 2.208773
I0110 18:38:05.530170 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-112"
I0110 18:38:08.931584 20252 trainer.py:126] Step #113: rate 0.001000, accuracy 36.0%, cross entropy 2.197516
I0110 18:38:08.931584 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-

I0110 18:40:18.103390 20252 trainer.py:126] Step #150: rate 0.001000, accuracy 34.0%, cross entropy 2.431519
I0110 18:40:18.103390 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-150"
I0110 18:40:21.306471 20252 trainer.py:126] Step #151: rate 0.001000, accuracy 31.0%, cross entropy 2.355087
I0110 18:40:21.307468 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-151"
I0110 18:40:24.592630 20252 trainer.py:126] Step #152: rate 0.001000, accuracy 32.0%, cross entropy 2.533646
I0110 18:40:24.593629 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-152"
I0110 18:40:27.766053 20252 trainer.py:126] Step #153: rate 0.001000, accuracy 34.0%, cross entropy 2.307920
I0110 18:40:27.768048 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-153"
I0110 18:40:33.986208 20252 trainer.py:126] Step #154: rate 0.001000, accuracy 34.0%, cross entropy 2.392353
I0110 18:40:34.001829 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-

...

I0110 19:15:21.772419 20252 trainer.py:126] Step #847: rate 0.001000, accuracy 67.0%, cross entropy 1.074740
I0110 19:15:21.773364 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-847"
I0110 19:15:24.575795 20252 trainer.py:126] Step #848: rate 0.001000, accuracy 72.0%, cross entropy 0.953894
I0110 19:15:24.575795 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-848"
I0110 19:15:27.411839 20252 trainer.py:126] Step #849: rate 0.001000, accuracy 63.0%, cross entropy 1.408028
I0110 19:15:27.411839 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-849"
I0110 19:15:30.534831 20252 trainer.py:126] Step #850: rate 0.001000, accuracy 65.0%, cross entropy 1.157611
I0110 19:15:30.536826 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-850"
I0110 19:15:33.494930 20252 trainer.py:126] Step #851: rate 0.001000, accuracy 55.0%, cross entropy 1.588382
I0110 19:15:33.495928 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-

I0110 19:17:22.521297 20252 trainer.py:126] Step #888: rate 0.001000, accuracy 63.0%, cross entropy 1.309475
I0110 19:17:22.522294 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-888"
I0110 19:17:25.724779 20252 trainer.py:126] Step #889: rate 0.001000, accuracy 68.0%, cross entropy 1.199175
I0110 19:17:25.725776 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-889"
I0110 19:17:28.903740 20252 trainer.py:126] Step #890: rate 0.001000, accuracy 65.0%, cross entropy 1.165608
I0110 19:17:28.904737 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-890"
I0110 19:17:31.736942 20252 trainer.py:126] Step #891: rate 0.001000, accuracy 63.0%, cross entropy 1.176130
I0110 19:17:31.738934 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-891"
I0110 19:17:34.762359 20252 trainer.py:126] Step #892: rate 0.001000, accuracy 70.0%, cross entropy 0.967493
I0110 19:17:34.763358 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-

I0110 19:19:28.457202 20252 trainer.py:126] Step #929: rate 0.001000, accuracy 65.0%, cross entropy 1.292135
I0110 19:19:28.458200 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-929"
I0110 19:19:31.378395 20252 trainer.py:126] Step #930: rate 0.001000, accuracy 66.0%, cross entropy 1.295759
I0110 19:19:31.379392 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-930"
I0110 19:19:34.421427 20252 trainer.py:126] Step #931: rate 0.001000, accuracy 64.0%, cross entropy 1.213113
I0110 19:19:34.422423 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-931"
I0110 19:19:37.342617 20252 trainer.py:126] Step #932: rate 0.001000, accuracy 65.0%, cross entropy 1.281789
I0110 19:19:37.343615 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-932"
I0110 19:19:40.310540 20252 trainer.py:126] Step #933: rate 0.001000, accuracy 70.0%, cross entropy 1.077110
I0110 19:19:40.312534 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-

I0110 19:21:27.473506 20252 trainer.py:126] Step #970: rate 0.001000, accuracy 55.0%, cross entropy 1.369135
I0110 19:21:27.473506 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-970"
I0110 19:21:30.596942 20252 trainer.py:126] Step #971: rate 0.001000, accuracy 62.0%, cross entropy 1.433916
I0110 19:21:30.598937 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-971"
I0110 19:21:33.636026 20252 trainer.py:126] Step #972: rate 0.001000, accuracy 58.0%, cross entropy 1.380152
I0110 19:21:33.637024 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-972"
I0110 19:21:36.675651 20252 trainer.py:126] Step #973: rate 0.001000, accuracy 62.0%, cross entropy 1.283381
I0110 19:21:36.675651 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-973"
I0110 19:21:39.524533 20252 trainer.py:126] Step #974: rate 0.001000, accuracy 66.0%, cross entropy 1.148832
I0110 19:21:39.525530 20252 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-

KeyboardInterrupt: 

In [14]:
# save_path=os.path.join(FLAGS.models_dir, model_architecture, '%s.pb'%os.path.basename(FLAGS.train_dir))
# freeze.freeze_graph(FLAGS,model_architecture,FLAGS.train_dir,save_path)